In [12]:
!pip3 install duckduckgo_search


[notice] A new release of pip is available: 23.0.1 -> 23.3.2
[notice] To update, run: C:\Users\blcke\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [13]:
import json
def safe_google_results(results: str | list) -> str:
    """
        Return the results of a Google search in a safe format.

    Args:
        results (str | list): The search results.

    Returns:
        str: The results of the search.
    """
    if isinstance(results, list):
        safe_message = json.dumps(
            [result.encode("utf-8", "ignore").decode("utf-8") for result in results]
        )
    else:
        safe_message = results.encode("utf-8", "ignore").decode("utf-8")
    return safe_message

In [14]:

import time
from itertools import islice

from duckduckgo_search import DDGS
def web_search(query: str, num_results: int = 8) -> str:
    """Return the results of a Google search

    Args:
        query (str): The search query.
        num_results (int): The number of results to return.

    Returns:
        str: The results of the search.
    """
    search_results = []
    attempts = 0

    while attempts < 3:
        if not query:
            return json.dumps(search_results)
        with DDGS(timeout=20) as ddgs:
            results = ddgs.text(query)
            search_results = list(islice(results, num_results))

            if search_results:
                break

            time.sleep(1)
            attempts += 1

    search_results = [
        {
            "title": r["title"],
            "url": r["href"],
            **({"exerpt": r["body"]} if r.get("body") else {}),
        }
        for r in search_results
    ]

    results = (
        "## Search results\n"
        # "Read these results carefully."
        # " Extract the information you need for your task from the list of results"
        # " if possible. Otherwise, choose a webpage from the list to read entirely."
        # "\n\n"
    ) + "\n\n".join(
        f"### \"{r['title']}\"\n"
        f"**URL:** {r['url']}  \n"
        "**Excerpt:** " + (f'"{exerpt}"' if (exerpt := r.get("exerpt")) else "N/A")
        for r in search_results
    )
    return safe_google_results(results)

In [25]:
from duckduckgo_search import DDGS,AsyncDDGS

async with AsyncDDGS(proxies={"https": "http://127.0.0.1:7088","http": "http://127.0.0.1:7088"}) as ddgs:
    results = [r async for r in ddgs.text("AI Aagent", max_results=5)]
    print(results)

[{'title': 'What is an AI agent? | Zapier', 'href': 'https://zapier.com/blog/ai-agent/', 'body': "What are AI agents? Right now, interacting with AI follows a familiar ritual. You enter a prompt, and the AI model calculates a response based on the input. Every time you want a new output, you have to provide a prompt. There's always a human to start the process. AI agents work in a different way. They're designed to think and act independently."}, {'title': 'AI Agents - Easy With AI', 'href': 'https://easywithai.com/ai-agents/', 'body': 'AI Agents BabyAGI BabyAGI is an AI-powered agent that uses OpenAI and vector databases like Chroma and Weaviate to perform tasks. It simplifies task creation, prioritization, and execution SuperAGI SuperAGI is an open-source platform that enables anyone to build AI agents and automate workflows.'}, {'title': 'What is an AI Agent?', 'href': 'https://oneai.com/learn/ai-agent', 'body': 'Simon Harris · Aug 3, 2023 · 14 min read time AI GPT LLM ML AI Agent O